In [1]:
import pandas as pd

### import extra package
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

#### modelling package
from statsmodels.graphics.mosaicplot import mosaic
import statsmodels
from statsmodels.formula.api import logit
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


#### importing evaluation package 
from sklearn.metrics import f1_score, confusion_matrix, roc_auc_score, auc,\
precision_recall_curve, roc_curve, average_precision_score
from sklearn import metrics

### Data Preparation 

In [4]:
### load data 
user_df = pd.read_csv("users.csv")
sessions_df = pd.read_csv("sessions.csv").drop_duplicates()
iaps_df = pd.read_csv("iaps.csv")
spending_df = pd.read_csv("spendevents.csv").drop_duplicates()

spending_df["date"] = pd.to_datetime(spending_df["date"])
sessions_df["date"] = pd.to_datetime(sessions_df["date"])
user_df["install_date"] = pd.to_datetime(user_df["install_date"])
iaps_df["date"] = pd.to_datetime(iaps_df["date"])

In [5]:
users_iaps = pd.merge(user_df, iaps_df, on = "user_id")
users_iaps["days_to_first_purchase"] = users_iaps["date"] - users_iaps["install_date"]
users_iaps["days_to_first_purchase"] = users_iaps["days_to_first_purchase"].dt.days
users_iaps["rank"] = users_iaps.groupby(["user_id"])["days_to_first_purchase"].rank(method="first", ascending=True)
users_iaps["rank"] = users_iaps["rank"].astype(int)
paying_users_df = users_iaps[users_iaps["rank"]==1]
first_iap_day_df = paying_users_df[["user_id", "ts", "days_to_first_purchase",\
                                    "install_date"]].rename(columns={"ts": "iap_date"})
first_iap_day_df.head()

TypeError: unsupported operand type(s) for -: 'str' and 'str'